In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 130)
pd.set_option('display.max_rows', 15000)
#pd.set_option('precision', 2)

In [25]:
df.sample(3)

,price_haul,price_price_deals,price_tons_over_3,customer_tons_include_price,customer_days_include,customer_container_size,customer_waste,customer_state,date_entered_year,short_zip,date_entered_month,date_entered_day,date_entered_hour,pred_haul,diff_haul
2410,150.00,HT,40.0,1.000000,10.0,10.0,Household,IA,2018.0,524,Jun,8.0,11.0,153.7750,-3.7750
33897,187.00,HT,52.0,2.000000,7.0,40.0,CandD,CA,2016.0,930,Feb,5.0,11.0,189.8198,-2.8198
23796,392.48,HWT,80.0,2.173913,7.0,20.0,CandD,CA,2018.0,956,Oct,22.0,13.0,429.0168,-36.5368


# Filtered Data Model

In [6]:
df=pd.read_csv("08_05_df_preds_fitered_data.csv",index_col=0)
df.shape

(25134, 15)

In [10]:
df_model = df[["price_haul","price_price_deals","price_tons_over_3",
                'customer_tons_include_price','customer_days_include', 'customer_container_size',
                'customer_waste','customer_state', 'date_entered_year','short_zip',
                'date_entered_month', 'date_entered_day', 'date_entered_hour']]
#df_model = df_model.dropna()
df_model.shape

(25134, 13)

In [ ]:
# "price_tons_over_3" ; 

X:              ["price_price_deals","price_tons_over_3",
                'customer_tons_include_price','customer_days_include', 'customer_container_size',
                'customer_waste','customer_state', 'date_entered_year','short_zip',
                'date_entered_month', 'date_entered_day', 'date_entered_hour']

y:               ["price_haul"]

In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics


In [15]:
# create a list of categorical variables for label encoding
cat_var = [ 'customer_waste',"price_price_deals","price_tons_over_3",
       'customer_state', 'short_zip', 
       'date_entered_month']

# create new encoded df with get_dummies

df_enc = pd.get_dummies(df_model, columns=cat_var)
X = df_enc.drop("price_haul",axis=1)
y = df_enc["price_haul"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

regressor = RandomForestRegressor(n_estimators=200, random_state=33)

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test) 


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 
print('Random Forest R2": %.4f' % r2_score(y_test, regressor.predict(X_test)))
print('Random Forest R2: %.4f' % regressor.score(X_test, y_test))

Mean Absolute Error: 24.24697498528008
Mean Squared Error: 1575.4758107763082
Root Mean Squared Error: 39.69226386559865
Random Forest R2": 0.8776
Random Forest R2: 0.8776


# Pickle Save/Load/Run

In [16]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model_haul = pickle.dumps(regressor) 
  
# Load the pickled model 
model_haul_from_pickle = pickle.loads(saved_model_haul) 
  
# Use the loaded pickled model to make predictions 
model_haul_from_pickle.predict(X_test)

print('Random Forest R2: %.4f' % r2_score(y_test, model_haul_from_pickle.predict(X_test)))

# save the model to disk
filename = 'model_haul_3cat.sav'
#pickle.dump(regressor, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
haul_loaded_model = pickle.load(open(filename, 'rb'))
result = haul_loaded_model.score(X_test, y_test)
print(result)

Random Forest R2: 0.8776
0.8776228841314929


# Single Prediction Code

In [19]:
# Enter values to get single prediction

input_dict = {
 'short_zip':[432] ,
 'customer_container_size': [30.0],
 'customer_days_include': [7.0],
 'customer_state': ['OH'],
 'customer_tons_include_price': [3.5],
 'customer_waste': ['CandD'],
 'date_entered_day': [19.0],
 'date_entered_hour': [17],
 'date_entered_month': ['Jun'],
 'date_entered_year': [2018.0],
 'price_price_deals': ['FF'],
  "price_tons_over_3":[75]}

cat_var = ['price_price_deals', 'customer_waste',
       'customer_state', 'date_entered_year',
       'date_entered_month', 'date_entered_day', 'date_entered_hour']

input_dict_df = pd.DataFrame.from_dict(input_dict)

df_input = pd.get_dummies(input_dict_df, columns=cat_var)

test_df = pd.DataFrame()

for col in X_test.columns:
    if col in df_input.columns:
        test_df[col] = df_input[col].values
    else:
        test_df[col] = 0


pred_haul = haul_loaded_model.predict(test_df)

if input_dict['price_price_deals'] == "FF":
    print("Predicted *price_total_cost* for input values:", pred_haul)
else:
    print("Predicted *price_haul* for input values:", pred_haul)

Predicted *price_haul* for input values: [345.40815]
